# Analyze Snai2 Staining Intensity

## Import Modules

In [1]:
# Import data handling and analysis packages
import os
import glob
import pandas as pd
from scipy import stats

# Import plotting packages
import iqplot
import bokeh.io
from bokeh.io import output_file, show
from bokeh.layouts import column, row
bokeh.io.output_notebook()

Loading BokehJS ...

## Import and parse raw data

In [22]:
# Navigate to CSV path
path = os.path.abspath('')+'/raw_data_csvs/'
full_df = pd.DataFrame()
list_ = []

for file_ in glob.glob(path + "/*.csv"):         # For loop to bring in files and concatenate them into a single dataframe
    df = pd.read_csv(file_)
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]                      # Determine Image name from file name
    df['Stain'], df['ROI'] = zip(*df['Label'].map(lambda x: x.split(':')))          # Split values in ROI label
    (df['ExptDate'], df['Treatment'], df['Dose'], df['Stains'], df['Embryo'],                   # Split values in Image name column
        df['Somites'], df['Section']) = zip(*df['Image'].map(lambda x: x.split('_')))
    df['EmbID'] = df['ExptDate'] + '_' + df['Stains'] + '_' + df['Embryo'] + '_' + df['Somites']
    list_.append(df)

full_df = pd.concat(list_)

# Isolate Snai2 stains (not using H2BRFP)
full_df = full_df.loc[full_df['Stain'] == 'Snai2']

full_df.head()

,,Label,Area,Mean,IntDen,RawIntDen,Image,Stain,ROI,ExptDate,Treatment,Dose,Stains,Embryo,Somites,Section,EmbID
5,6,Snai2:background,19.787,28.695,567.798,11019.0,20210824_SMPD3FLAG_3ugul_BF;Pax7;H2BRFP;TCFLef...,Snai2,background,20210824,SMPD3FLAG,3ugul,BF;Pax7;H2BRFP;TCFLefH2Bd2EGFP;Snai2,Emb4,8ss,sec1,20210824_BF;Pax7;H2BRFP;TCFLefH2Bd2EGFP;Snai2_...
6,7,Snai2:background,27.516,28.391,781.231,15161.0,20210824_SMPD3FLAG_3ugul_BF;Pax7;H2BRFP;TCFLef...,Snai2,background,20210824,SMPD3FLAG,3ugul,BF;Pax7;H2BRFP;TCFLefH2Bd2EGFP;Snai2,Emb4,8ss,sec1,20210824_BF;Pax7;H2BRFP;TCFLefH2Bd2EGFP;Snai2_...
7,8,Snai2:background,18.705,27.251,509.725,9892.0,20210824_SMPD3FLAG_3ugul_BF;Pax7;H2BRFP;TCFLef...,Snai2,background,20210824,SMPD3FLAG,3ugul,BF;Pax7;H2BRFP;TCFLefH2Bd2EGFP;Snai2,Emb4,8ss,sec1,20210824_BF;Pax7;H2BRFP;TCFLefH2Bd2EGFP;Snai2_...
8,9,Snai2:Cntl,4622.306,312.556,1444730.785,28037237.0,20210824_SMPD3FLAG_3ugul_BF;Pax7;H2BRFP;TCFLef...,Snai2,Cntl,20210824,SMPD3FLAG,3ugul,BF;Pax7;H2BRFP;TCFLefH2Bd2EGFP;Snai2,Emb4,8ss,sec1,20210824_BF;Pax7;H2BRFP;TCFLefH2Bd2EGFP;Snai2_...
9,10,Snai2:Expt,3066.851,371.166,1138310.701,22090681.0,20210824_SMPD3FLAG_3ugul_BF;Pax7;H2BRFP;TCFLef...,Snai2,Expt,20210824,SMPD3FLAG,3ugul,BF;Pax7;H2BRFP;TCFLefH2Bd2EGFP;Snai2,Emb4,8ss,sec1,20210824_BF;Pax7;H2BRFP;TCFLefH2Bd2EGFP;Snai2_...


## Calculate Corrected Total Cellular Fluorescence (CTCF) for each ROI

This will determine the fluorescence in each ROI (expression level), corrected for size and background intensity in the image, and perform this analysis for each stain in the dataset

In [23]:
# Get a list of treatments and stains
treatment_list = full_df.Treatment.unique().tolist()
stain_list = full_df.Stain.unique().tolist()

# Mean background values and group by Treatment, Embryo, Fluor, ROI and Section
mean_sections = ((full_df.groupby(['Stain', 'Treatment', 'Embryo', 'EmbID', 'ROI', 'Section', 'ExptDate'])
                  ['Area', 'Mean', 'IntDen']).mean())

# Loop through stains, performing the following analysis
for j in stain_list:
    stain = j
    df_stain = pd.DataFrame(mean_sections.xs(stain))
    
    # Loop trough treatments, performing each analysis and exporting CSV file for each treatment
    for i in treatment_list:
        # Slice dataframe to process only embryos with given treatment
        treatment = i
        df_treatment = pd.DataFrame(df_stain.xs(treatment))

        # Determine CTCF values = ROI IntDen - (background mean * ROI area)
        # Calculate background (background mean * ROI area)
        background_corr_cntl = (df_treatment.xs('background', level='ROI')['Mean'] 
                        * df_treatment.xs('Cntl', level='ROI')['Area'])
        background_corr_expt = (df_treatment.xs('background', level='ROI')['Mean'] 
                        * df_treatment.xs('Expt', level='ROI')['Area'])

        # Slice out only Cntl or Expt values in IntDen
        intdens_cntl = df_treatment.xs('Cntl', level='ROI')['IntDen'] 
        intdens_expt = df_treatment.xs('Expt', level='ROI')['IntDen'] 

        # Subtract background from IntDens to determine CTCF and concatenate into single dataframe
        sub_cntl = pd.DataFrame(intdens_cntl - background_corr_cntl)
        sub_expt = pd.DataFrame(intdens_expt - background_corr_expt)
        full_ctcf = pd.concat([sub_cntl, sub_expt], keys = ['Cntl', 'Expt'])
        full_ctcf.columns = ['CTCF']

        # Combine raw values, generate ratio
        ctcf_cntl = full_ctcf.xs('Cntl').reset_index()
        ctcf_cntl.rename(columns={'CTCF':'Cntl CTCF'}, inplace=True)
        ctcf_expt = full_ctcf.xs('Expt').reset_index()
        ctcf_expt.rename(columns={'CTCF':'Expt CTCF'}, inplace=True)
        results = pd.concat([ctcf_cntl,ctcf_expt], axis=1)
        results['Expt/Cntl CTCF'] = ctcf_expt['Expt CTCF'] / ctcf_cntl['Cntl CTCF']
        results = results.loc[:,~results.columns.duplicated()]
        results = results.groupby(['Embryo', 'ExptDate', 'EmbID']).mean().reset_index()

        # Normalize all CTCF values to mean of control group
        norm_cntl = pd.DataFrame(results['Cntl CTCF']/(float(results['Cntl CTCF'].mean())))
        norm_cntl.rename(columns={'Cntl CTCF':'Norm Cntl CTCF'}, inplace=True)
        norm_expt = pd.DataFrame(results['Expt CTCF']/(float(results['Cntl CTCF'].mean())))
        norm_expt.rename(columns={'Expt CTCF':'Norm Expt CTCF'}, inplace=True)
        norm_expt.columns = ['Norm Expt CTCF']
        results = pd.concat([results, norm_cntl, norm_expt], axis=1, sort=False)
        results.to_csv(stain + '_' + treatment + '_CTCF.csv')
        
results

,Embryo,ExptDate,EmbID,Cntl CTCF,Expt CTCF,Expt/Cntl CTCF,Norm Cntl CTCF,Norm Expt CTCF
0,Emb1,20210824,20210824_BF;Pax7;H2BRFP;BREH2Bd2EGFP;Snai2_Emb...,9.034916e+05,4.067338e+05,0.450180,1.124905,0.506410
1,Emb1,20210830,20210830_BF;Pax7;H2BRFP;BREH2Bd2EGFP;Snai2_Emb...,1.034452e+06,1.174323e+06,1.229714,1.287959,1.462108
2,Emb1,20210830,20210830_BF;Pax7;H2BRFP;TCFLefH2Bd2EGFP;Snai2_...,6.412530e+05,6.443185e+05,1.004781,0.798401,0.802218
3,Emb2,20210830,20210830_BF;Pax7;H2BRFP;TCFLefH2Bd2EGFP;Snai2_...,6.124221e+05,7.211164e+05,1.177482,0.762505,0.897836
4,Emb3,20210824,20210824_BF;Pax7;H2BRFP;TCFLefH2Bd2EGFP;Snai2_...,7.100314e+05,4.595928e+05,0.653131,0.884035,0.572223
5,Emb3,20210830,20210830_BF;Pax7;H2BRFP;BREH2Bd2EGFP;Snai2_Emb...,7.961922e+05,7.410367e+05,0.949004,0.991311,0.922639
6,Emb3,20210830,20210830_BF;Pax7;H2BRFP;TCFLefH2Bd2EGFP;Snai2_...,5.033492e+05,4.720316e+05,0.950915,0.626702,0.587710
7,Emb4,20210824,20210824_BF;Pax7;H2BRFP;TCFLefH2Bd2EGFP;Snai2_...,1.133085e+06,7.745209e+05,0.681330,1.410764,0.964329
8,Emb4,20210830,20210830_BF;Pax7;H2BRFP;TCFLefH2Bd2EGFP;Snai2_...,5.695799e+05,4.296000e+05,0.754240,0.709164,0.534880
9,Emb5,20210824,20210824_BF;Pax7;H2BRFP;BREH2Bd2EGFP;Snai2_Emb...,6.047865e+05,6.101940e+05,0.991401,0.752998,0.759731


## Plot and perform statistical analysis - Snai2

In [24]:
data = results.reset_index()

# Build Stripbox plot
stripbox = iqplot.stripbox(
                    # Data to plot
                        data=data,
                        q='Expt/Cntl CTCF', q_axis='y',

                    # Plot details
                        jitter=True, jitter_kwargs=dict(width=0.3),
                        marker_kwargs=dict(alpha=0.8, size=8
#                                            ,color='darkgray'
                                          ),
                        box_kwargs=dict(line_color='black', line_width=1.5),
                        whisker_kwargs=dict(line_color='black', line_width=1.5),
                        median_kwargs=dict(line_color='black', line_width=2),
                        top_level='box',
                        frame_width=150, frame_height=300,

                    # Plot customizations
#                         order=targets,
                        y_range=(0,2),
                        y_axis_label='Normalized CTCF (Expt/Cntl)',
                        show_legend=False,
)

# Final customizations
stripbox.axis.axis_label_text_font_size = '16px'
stripbox.axis.major_label_text_font_size = '16px'
stripbox.axis.axis_label_text_font_style = 'bold'
stripbox.xaxis.major_label_text_font_style = 'italic'

# View plot
show(stripbox)

In [25]:
################### Isolate data for analysis ###################
# Pull out only cells and treaments of interest, and rename ROIs with the appropriate constructs
data = results.reset_index()
data=data.filter(['EmbID', 'Norm Cntl CTCF', 'Norm Expt CTCF'])
data=data.melt(id_vars=['EmbID'], var_name='ROI', value_name='Norm CTCF')

################### Plot as strip plot ###################
# Plot as strip plot
p1 = iqplot.strip(data=data
                ,q='Norm CTCF', q_axis='y'
                ,cats=['ROI'], parcoord_column='EmbID'
                ,y_range=(0,3.5)
                ,frame_height = 300, frame_width = 150
                ,y_axis_label= 'Normalized CTCF'
#                 ,color_column='Image'
                ,marker_kwargs=dict(size=5,color='black')
                ,parcoord_kwargs=dict(line_width=1,color='gray')
#                 ,show_legend=True
              )

# p1.axis.axis_label_text_font_style = 'bold italic'
p1.axis.axis_label_text_font_size = '14px'
p1.axis.major_label_text_font_size = '12px'
p1.axis.axis_label_text_font_style = 'normal'
p1.xaxis.major_label_orientation = 7

show(row(p1))

################### Perform statistical analysis ###################

# Perform Paired t test 
cntl = data.loc[data['ROI'] == 'Norm Cntl CTCF']['Norm CTCF']
expt = data.loc[data['ROI'] == 'Norm Expt CTCF']['Norm CTCF']
ttest = stats.ttest_rel(cntl,expt)

# Display test results
print('Paired t-test results: \n\t\t statistic=' + str(ttest[0]) + 
    '\n\t\t p-value=' + str(ttest[1]))

Paired t-test results: 
		 statistic=2.742704452351329
		 p-value=0.01676995746022817
